<a href="https://colab.research.google.com/github/Alaaokaly/nlp-foundations/blob/main/EncoderDecoderModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Node ():
    def multi(self,x):
        print(x*x*2)

node = Node()
node.multi(2)

8


In [51]:
# encoder architucture for seq2seq structure
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, num_hidden, num_layers, drop_out = 0 ):

       super().__init__()
       self.embedding = nn.Embedding(vocab_size, embedding_size)
       self.encoder = nn.GRU(embedding_size, num_hidden, num_layers, dropout = drop_out)
    def forward(self, X):
        embds = self.embedding(X.t())
        outputs, hidden_state = self.encoder(embds)
        return outputs, hidden_state


vocab_size, embed_size, num_hiddens, num_layers = 10, 8, 16, 2
batch_size, num_steps = 4, 9
encoder = Encoder(vocab_size, embed_size, num_hiddens, num_layers)
X = torch.zeros((batch_size,num_steps), dtype=torch.long)
outputs, state = encoder(X)
print(outputs.shape)
print(state.shape)




torch.Size([9, 4, 16])
torch.Size([2, 4, 16])


In [52]:
""" it's important to make sure that the dim of both the encoder and decoder match
as in having the same layer_num and num_hidden
this ensures that the info transfer at each step """

" it's important to make sure that the dim of both the encoder and decoder match \nas in having the same layer_num and num_hidden \nthis ensures that the info transfer at each step "

In [63]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, num_hiddens, num_layers, drop_out=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.decoder = nn.GRU(embedding_size + num_hiddens, num_hiddens, num_layers, dropout=drop_out)
        self.dense = nn.LazyLinear(vocab_size)

    def init_state(self, encoder_state):
        # Return the last hidden state for the decoder
        return encoder_state[-1]

    def forward(self, X, state):
        embeds = self.embedding(X)  # Shape: (batch_size, num_steps, embedding_size)

        # Get the last hidden state and reshape for context
        context = state[-1].unsqueeze(1)  # Shape: (batch_size, 1, num_hidden)
        context = context.repeat(1, embeds.size(1), 1)  # Shape: (batch_size, num_steps, num_hidden)

        print("Shape of embeds:", embeds.shape)  # Should be (4, 9, 8)
        print("Shape of context:", context.shape)  # Should be (4, 9, 16)

        # Concatenate the embeddings and the context
        embs_and_context = torch.cat((embeds, context), dim=-1)  # Shape: (batch_size, num_steps, embedding_size + num_hidden)

        # Use GRU to generate the decoder output
        outputs, hidden_state = self.decoder(embs_and_context.transpose(0, 1), state)  # Transpose for GRU: (num_steps, batch_size, input_size)

        outputs = self.dense(outputs).swapaxes(0, 1)  # Shape: (batch_size, num_steps, vocab_size)

        return outputs, hidden_state  # Return outputs and new hidden state


encoder = Encoder(vocab_size, embed_size, num_hiddens, num_layers)
outputs, state = encoder(X)



decoder = Decoder(vocab_size, embed_size, num_hiddens, num_layers)
dec_outputs, new_state = decoder(X, state)

print("Decoder outputs shape:", dec_outputs.shape)  # Expected: (batch_size, num_steps, vocab_size)


Shape of embeds: torch.Size([4, 9, 8])
Shape of context: torch.Size([4, 9, 16])
Decoder outputs shape: torch.Size([4, 9, 10])
